In [1]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_model

In [2]:
daxdata= get_DAX.get()

In [3]:
baseline_df = DAX_baseline.DAX_baseline(daxdata)

In [4]:
baseline_df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-30,DAX,1 day,-3.321265,-0.546439,0.068631,0.718274,2.719772
1,2023-11-30,DAX,2 day,-4.087895,-0.817404,0.080265,1.000970,3.570492
2,2023-11-30,DAX,5 day,-5.259161,-0.912912,0.164214,1.232426,4.628697
3,2023-11-30,DAX,6 day,-6.282629,-1.044070,0.203116,1.443441,5.367396
4,2023-11-30,DAX,7 day,-6.735742,-1.091646,0.343478,1.594461,6.259934


In [5]:
quantile_df = DAX_quantile_regression.DAX_quantile_regression(daxdata)

In [6]:
quantile_df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-30,DAX,1 day,-2.146732,-0.480537,0.001527,0.508445,1.686040
1,2023-11-30,DAX,2 day,-0.239467,1.412230,1.998202,2.504450,4.035406
2,2023-11-30,DAX,5 day,-0.089087,1.632409,2.261871,2.718899,4.064714
3,2023-11-30,DAX,6 day,-0.967677,0.827741,1.499534,1.980733,3.251140
4,2023-11-30,DAX,7 day,-0.513234,1.159863,1.796705,2.205429,3.696325


In [7]:
baseline_model = {
    'name': 'Baseline Model',
    'function': DAX_baseline.DAX_baseline
}
quantile_model = {
    'name': 'Quantile Regression',
    'function': DAX_quantile_regression.DAX_quantile_regression
}


In [8]:
start_date = '2023-01-01'
end_date = '2023-01-31'

evaluation_model1, evaluation_model2 = evaluate_model.evaluate(baseline_model, quantile_model, daxdata, start_date, end_date)

AttributeError: 'DatetimeIndex' object has no attribute 'dt'

In [ ]:
evaluation_model1['score'].sum()

In [ ]:
evaluation_model2['score'].sum()

In [ ]:
evaluation_model1['score'].mean()

In [ ]:
evaluation_model2['score'].mean()

In [ ]:
evaluation_model1['score'].std()

In [ ]:
evaluation_model2['score'].std()

In [ ]:
from functions import naive_ensemble

In [ ]:
evaluation_model1

In [ ]:
evaluation_model2

In [ ]:
ensemble_df = naive_ensemble.ensemble(evaluation_model1, evaluation_model2,0.5,0.5)

In [ ]:
ensemble_df

In [ ]:
import pandas as pd
def evaluate_ensemble_model(ensemble_df, actual_values):
    """
    Evaluate the ensemble model using the provided scoring method.

    :param ensemble_df: DataFrame containing the ensemble forecasts
    :param actual_values: Series or DataFrame containing the actual observed values
    :return: DataFrame with the scores for each horizon in the ensemble forecast
    """
    scores = []
    for index, row in ensemble_df.iterrows():
        forecast_date = row['forecast_date']
        horizon = row['horizon']
        quantile_preds = [row[f'q{q}'] for q in ['0.025', '0.25', '0.5', '0.75', '0.975']]
        obs = actual_values.loc[(actual_values['forecast_date'] == forecast_date) & (actual_values['horizon'] == horizon), 'ret1'].iloc[0]
        score = evaluate_horizon(quantile_preds, obs)
        scores.append({'forecast_date': forecast_date, 'horizon': horizon, 'score': score})

    return pd.DataFrame(scores)

# Example usage
# ensemble_df is your DataFrame containing the ensemble forecasts
# actual_values is your DataFrame containing the actual observed values (e.g., 'Close' prices)
ensemble_scores = evaluate_ensemble_model(ensemble_df, evaluation_model1)
print(ensemble_scores)


In [ ]:
ensemble_scores['score'].sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=ensemble_scores, x='forecast_date', y='score')
plt.title('Model Score Over Time')
plt.xlabel('Date')
plt.ylabel('Score')
plt.xticks(rotation=45)  # Optional: Rotate x-axis labels for better readability
plt.show()

In [ ]:
quantile_model=dax_quantile_regression.dax_quantile_regression(daxdata)

In [ ]:
ensemble_df=naive_ensemble.ensemble(evaluation_model1, evaluation_model2,0.5,0.5)

NameError: name 'energy_quantile_regression' is not defined